In [1]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00


In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.6 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

In [28]:
# adapter_id = "mridul161203/qwen-lora"
# model.load_adapter(adapter_id)

adapter_config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.8M [00:00<?, ?B/s]

In [4]:
import time

def generate_output(messages):
  print(messages)
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  start_time = time.time()
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  latency = time.time() - start_time

  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  print(response)

  return response, latency

In [5]:
from tqdm import tqdm
import re

def format_data(system, prompt, answer):
    return {"messages": [
                {
                    "role": "system",
                    "content": system
                },
                {
                    "role": "user",
                    "content": prompt
                },
                {
                    "role": "assistant",
                    "content": answer
                }
            ]
        }

def convert_camel_case_to_snake_case(word):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', word).lower()

def format_json_string(json_str):
    return f"```json\n{json_str}\n```"

def prepare_data_for_llama(df):

    formatted_dataset = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        system = row["instruction"]
        prompt = row["input"]
        answer = row["output"]
        # data_list = json.loads(data_str)
        # data_list = [{k: v for d in data_list for k, v in d.items()}]
        # answer = format_json_string(data_list[0])
        formatted_dataset.append(format_data(system, prompt, answer))

    return formatted_dataset


In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset

dataset_id = "llamafactory/alpaca_en"

dataset = load_dataset(dataset_id)


df = dataset["train"].to_pandas()[:10]

In [9]:
df

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced diet and make sure to includ...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,"An atom is made up of a nucleus, which contain..."
3,How can we reduce air pollution?,,There are a number of ways to reduce air pollu...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...
5,"Write a concise summary of the following:\n""Co...",,The Commodore 64 was a highly successful 8-bit...
6,Explain why the following fraction is equivale...,4/16,The fraction 4/16 is equivalent to 1/4 because...
7,Write a short story in third person narration ...,,John was at a crossroads in his life. He had j...
8,Evaluate this sentence for spelling and gramma...,He finnished his meal and left the resturant,He finished his meal and left the restaurant.
9,How did Julius Caesar die?,,Julius Caesar was assassinated by a group of u...


In [13]:
data = prepare_data_for_llama(df)

100%|██████████| 10/10 [00:00<00:00, 6223.00it/s]


In [ ]:
import mlflow
import torch
import pandas as pd

mlflow.set_experiment("qwen_base_validations")

with mlflow.start_run():

  evaluation_results = []

  for idx, row_data in tqdm(enumerate(data), total=len(data)):
        print(f"Processing row {idx + 1}/{len(data)} - Fine-tuned Model")

        messages = row_data["messages"]
        messages = [message for message in row_data['messages'] if message['role'] != 'assistant']

        finetuned_model_output, latency = generate_output(messages)

        evaluation_results.append({
            "row_index": idx + 1,
            "system": row_data["messages"][0],
            "prompt": row_data["messages"][1],
            "ground_truth": row_data["messages"][2],
            "finetuned_model_output": finetuned_model_output,
            "latency": latency,
            # "similarity_ratio": similarity_ratio,
            # "cosine_similarity": cos_similarity,
        })

        torch.cuda.empty_cache()

  evaluation_df = pd.DataFrame(evaluation_results)

  json_path = "evaluation_results_finetuned.json"
  evaluation_df.to_json(json_path, orient="records", lines=True)
  mlflow.log_artifact(json_path, "tables")

  mlflow.log_param("training epochs", 3)
  mlflow.log_param("batch_size", 1)
  mlflow.log_param("learning_rate", 2e-4)
  mlflow.log_param("max_tokens", 8192)
  mlflow.log_param("top_p", 0.6)
  mlflow.log_param("temperature", 0.9)
  mlflow.log_param("top_k", 50)
  mlflow.log_param("task_type", "text generation")
  mlflow.log_param("num_rows", len(data))

  mlflow.log_param("average_latency", evaluation_df["latency"].mean())
  mlflow.log_table(evaluation_df, "evaluation_results.json")

  mlflow.set_tags({
  "model": "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
  "lora-target": "q_proj,k_proj,v_proj,o_proj"
  })



  0%|          | 0/10 [00:00<?, ?it/s]

Processing row 1/10 - Fine-tuned Model
[{'role': 'system', 'content': 'Give three tips for staying healthy.'}, {'role': 'user', 'content': ''}]


 10%|█         | 1/10 [00:15<02:22, 15.86s/it]

1. Eat a balanced diet: Consuming a variety of nutrient-dense foods, such as fruits, vegetables, whole grains, lean protein and healthy fats, can help provide your body with the necessary nutrients to function at its best.

2. Exercise regularly: Engaging in regular physical activity, such as walking, running, swimming or yoga, can help improve overall health, reduce the risk of chronic diseases, and boost mood and energy levels.

3. Get enough sleep: Adequate sleep is essential for overall health and well-being. It helps the body repair and regenerate tissues, consolidate memories, and regulate hormones. Aim for 7-9 hours of quality sleep per night to help maintain optimal health.
Processing row 2/10 - Fine-tuned Model
[{'role': 'system', 'content': 'What are the three primary colors?'}, {'role': 'user', 'content': ''}]


 20%|██        | 2/10 [00:22<01:23, 10.41s/it]

...

An atom is the basic unit of matter and the smallest component of an element that retains all of the chemical properties of that element. Atoms have a unique structure consisting of three types of subatomic particles: protons, neutrons and electrons.

The protons and neutrons are located in the center of the atom, in a region called the nucleus. The protons have a positive electrical charge, while the neutrons are electrically neutral. The number of protons in the nucleus determines the atomic number of the element and defines its chemical properties.

Surrounding the nucleus is a cloud of electrons, which are negatively charged subatomic particles. These electrons orbit the nucleus in specific energy levels or shells. The number of electrons in an atom's outermost shell determines its reactivity and ability to form chemical bonds with other atoms.

In summary, the structure of an atom consists of a central nucleus containing protons and neutrons, surrounded by electrons that orbit in 

 40%|████      | 4/10 [01:21<02:25, 24.32s/it]

Air pollution is a major concern for the health of our planet and its inhabitants. There are several ways to reduce air pollution, some of which include:

1. Use of clean energy: One of the main sources of air pollution is the burning of fossil fuels such as coal, oil, and gas. Switching to clean and renewable sources of energy like wind, solar or hydro power can significantly reduce emissions.

2. Improved vehicle emission standards: Cars and trucks are another major source of air pollution. Governments can implement stricter emission standards for vehicles, and encourage the use of electric or hybrid cars that produce zero emissions.

3. Promoting public transportation: Encouraging the use of public transportation can help reduce the number of individual vehicles on the road, thereby reducing emissions. This could be done through the provision of affordable and efficient public transport systems.

4. Planting trees: Trees absorb carbon dioxide from the atmosphere, so planting more tr

 50%|█████     | 5/10 [02:03<02:32, 30.54s/it]

As a project manager for a construction company, I have faced many challenging situations throughout my career. One of the most difficult decisions I had to make was regarding the scope of work for a major construction project.

The project was behind schedule and over budget, and the client was becoming increasingly frustrated. After reviewing the situation with the project team, it became clear that we would not be able to meet our original deadline without cutting costs in some way. However, there were several options available, each with its own set of consequences.

One option was to reduce the scope of work by eliminating certain features or materials, which would save money but compromise the quality of the final product. Another option was to seek additional funding from the client, which could potentially keep the project on track but may not be feasible given the current economic climate. A third option was to reassign staff and resources to focus solely on completing the rem

In [15]:
!pip install pyngrok

In [16]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [20]:
!ngrok config add-authtoken $add_authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [27]:
from pyngrok import ngrok

ngrok.kill()

NGROK_AUTH_TOKEN = "$add_authtoken"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://37ae-34-125-145-92.ngrok-free.app
